# pyLogit Example
The purpose of this notebook is to demonstrate they key functionalities of pyLogit:
<ol>
    <li> Converting data between 'wide' and 'long' formats. </li>
    <li> Estimating conditional logit models. </li>
</ol>

The dataset being used for this example is the "Swissmetro" dataset used in the Python Biogeme examples. The data can be downloaded at <a href="http://biogeme.epfl.ch/examples_swissmetro.html">http://biogeme.epfl.ch/examples_swissmetro.html</a>, and a detailed explanation of the variables and data-collection procedure can be found at http://www.strc.ch/conferences/2001/bierlaire1.pdf.

Relevant information about this dataset is that it is from a stated preference survey about whether or not individuals would use a new underground Magnetic-Levetation train system called the Swissmetro. 

The overall set of possible choices in this dataset was "Train", "Swissmetro", and "Car." However, the choice set faced by each individual is <strong><u>not</u></strong> constant. An individual's choice set was partially based on the alternatives that he/she was capable of using at the moment. For instance, people who did not own cars did not receive a stated preference question where car was an alternative that they could choose. Note that because the choice set varies across choice situations, mlogit and statsmodels could not be used with this dataset. 

Also, each individual responded to multiple choice situations. Thus the choice observations are not truly independent of all other choice observations (they are correlated accross choices made by the same individual). However, for the purposes of this example, the effect of repeat-observations on the typical i.i.d. assumptions will be ignored.

Based on the Swissmetro data, we will build a travel mode choice model for individuals who are commuting or going on a business trip.

In [3]:
from collections import OrderedDict    # For recording the model specification 

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations

import pylogit as pl                   # For MNL model estimation and
                                       # conversion from wide to long format


## Load and filter the raw Swiss Metro data

In [29]:
trips = pd.read_csv(f"results/trips_pre_model.csv",index_col=0)
trips = trips.drop(columns=['O_lat', 'O_long', 'D_lat', 'D_long', 'times', 'Dur_Tot',
       'destination_coords', 'chosen_time', '30%', 'gm_ua', 'Mun_Ori', 'Mun_Des'])

trips['drive_av'] = 1
trips['transit_av'] = 1
trips['walk_av'] = 1

mapeo = {'Car': 1, 'PT': 2, 'Andando': 3}
trips['Modo'] = trips['Modo'].map(mapeo)

trips.rename(columns={'Hora_Ini_E': 'Departure time', 'Per_hog': 'HH size', 'Turismos': 'Nº cars/HH',
                   'Sexo': 'Gender', 'Edad': 'Age', 'crnt_tur': 'Driver\'s license',
                   'drive_tt': 'Car $T_t$', 'distance': 'Distance', 'walk_tt': 'Walk $T_t$',
                   'transit_tt_gm': 'PT $T_t$', 'Tipo_familia': 'Family type'}, inplace=True)

trips = trips[['Departure time', 'HH size', 'Nº cars/HH', 'Gender', 'Age', 'Driver\'s license', 'Family type', 'Distance', 'drive_av', 'transit_av', 'walk_av', 'Car $T_t$', 'PT $T_t$', 'Walk $T_t$', 'Modo']]
# trips = trips.rename(columns={'transit_tt_gm': 'transit_tt'})
trips.T

,0,1,2,3,4,5,6,7,8,9,...,10308,10309,10310,10311,10312,10313,10314,10315,10316,10317
Departure time,109.000000,62.00000,180.000000,198.000000,106.000000,103.000000,108.000000,107.000000,101.000000,69.000000,...,176.000000,180.000000,181.000000,182.000000,109.000000,83.000000,100.000000,159.000000,93.0000,108.000000
HH size,2.000000,2.00000,2.000000,3.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000,...,1.000000,3.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,1.0000,1.000000
Nº cars/HH,2.000000,2.00000,4.000000,2.000000,1.000000,1.000000,3.000000,2.000000,2.000000,1.000000,...,1.000000,3.000000,3.000000,3.000000,3.000000,1.000000,2.000000,2.000000,1.0000,2.000000
Gender,1.000000,1.00000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,...,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,1.0000,2.000000
Age,6.000000,4.00000,3.000000,5.000000,5.000000,6.000000,3.000000,4.000000,5.000000,4.000000,...,4.000000,5.000000,4.000000,5.000000,5.000000,4.000000,4.000000,5.000000,2.0000,6.000000
Driver's license,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.0000,1.000000
Family type,4.000000,2.00000,2.000000,2.000000,2.000000,3.000000,2.000000,6.000000,4.000000,3.000000,...,2.000000,4.000000,4.000000,4.000000,4.000000,2.000000,3.000000,3.000000,2.0000,4.000000
Distance,7713.093000,8962.58400,6449.426000,6794.456000,768.076000,5877.712000,10861.937000,6011.384000,2900.038000,3590.849000,...,1207.239000,291.519000,900.307000,409.071000,648.794000,1240.299000,495.243000,291.519000,862.1640,268.524000
drive_av,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.000000
transit_av,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0000,1.000000


## Convert the Swissmetro data to "Long Format"

pyLogit only estimates models using data that is in "long" format. 

Long format has 1 row per individual per available alternative, and wide format has 1 row per individual or observation. Long format is useful because it permits one to directly use matrix dot products to calculate the index, $V_{ij} = x_{ij} \beta$, for each individual $\left(i \right)$ for each alternative $\left(j \right)$. In applications where one creates one's own dataset, the dataset can usually be created in long format from the very beginning. However, in situations where a dataset is provided to you in wide format (as in the case of the Swiss Metro dataset), it will be necesssary to convert the data from wide format to long format.

To convert the raw swiss metro data to long format, we need to specify:
<ol>
    <li>the variables or columns that are specific to a given individual, regardless of what alternative is being considered (note: every row is being treated as a separate observation, even though each individual gave multiple responses in this stated preference survey)</li>
    <li>the variables that vary across some or all alternatives, for a given individual (e.g. travel time)</li>
    <li>the availability variables</li>
    <li>the <u>unique</u> observation id column. (Note this dataset has an observation id column, but for the purposes of this example we don't want to consider the repeated observations of each person as being related. We therefore want a identifying column that gives an id to every response of every individual instead of to every individual).</li>
    <li>the choice column</li>
</ol>
<br>The cells below will identify these various columns, give them names in the long-format data, and perform the necessary conversion. 



In [31]:
# Create the list of individual specific variables
ind_variables = trips.columns.tolist()[:8]

# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe.
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'Car $T_t$'),
                                               (2, 'PT $T_t$'),
                                               (3, 'Walk $T_t$')])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'drive_av',
                          2: 'transit_av', 
                          3: 'walk_av'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
trips[obs_id_column] = np.arange(trips.shape[0],
                                            dtype=int) + 1


# Create a variable recording the choice column
choice_column = "Modo"

In [32]:
# Perform the conversion to long-format
long_trips = pl.convert_wide_to_long(trips, 
                                    ind_variables, 
                                    alt_varying_variables, 
                                    availability_variables, 
                                    obs_id_column, 
                                    choice_column,
                                    new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_trips.head(10).T

,0,1,2,3,4,5,6,7,8,9
custom_id,1.000000,1.000,1.000000,2.0000,2.000,2.00000,3.000000,3.000,3.000000,4.000000
mode_id,1.000000,2.000,3.000000,1.0000,2.000,3.00000,1.000000,2.000,3.000000,1.000000
Modo,1.000000,0.000,0.000000,1.0000,0.000,0.00000,0.000000,1.000,0.000000,1.000000
Departure time,109.000000,109.000,109.000000,62.0000,62.000,62.00000,180.000000,180.000,180.000000,198.000000
HH size,2.000000,2.000,2.000000,2.0000,2.000,2.00000,2.000000,2.000,2.000000,3.000000
Nº cars/HH,2.000000,2.000,2.000000,2.0000,2.000,2.00000,4.000000,4.000,4.000000,2.000000
Gender,1.000000,1.000,1.000000,1.0000,1.000,1.00000,2.000000,2.000,2.000000,1.000000
Age,6.000000,6.000,6.000000,4.0000,4.000,4.00000,3.000000,3.000,3.000000,5.000000
Driver's license,1.000000,1.000,1.000000,1.0000,1.000,1.00000,1.000000,1.000,1.000000,1.000000
Family type,4.000000,4.000,4.000000,2.0000,2.000,2.00000,2.000000,2.000,2.000000,2.000000


## Perform desired variable creations and transformations
Before estimating a model, one needs to pre-compute all of the variables that one wants to use. This is different from the functionality of other packages such as mlogit or statsmodels that use formula strings to create new variables "on-the-fly." This is also somewhat different from Python Biogeme where new variables can be defined in the script but not actually created by the user before model estimation. pyLogit does not perform variable creation. It only estimates models using variables that already exist.

Below, we pre-compute the variables needed for this example's model:
<ol>
    <li> Travel time in hours instead of minutes. </li>
    <li> Travel cost in units of 0.01 CHF (swiss franks) instead of CHF, for ease of numeric optimization. </li>
    <li> Travel cost interacted with a variable that identifies individuals who own a season pass (and therefore have no marginal cost of traveling on the trip) or whose employer will pay for their commute/business trip. </li>
    <li> A dummy variable for traveling with a single piece of luggage. </li>
    <li> A dummy variable for traveling with multiple pieces of luggage. </li>
    <li> A dummy variable denoting whether an individual is traveling first class. </li>
    <li> A dummy variable indicating whether an individual took their survey on-board a train (since it is a-priori expected that these individuals are already willing to take a train or train-like service such as Swissmetro).</li>
</ol>

In [7]:
##########
# Create scaled variables so the estimated coefficients are of similar magnitudes
##########
# Scale the travel time column by 60 to convert raw units (minutes) to hours
# long_swiss_metro["travel_time_hrs"] = long_swiss_metro["travel_time"] / 60.0

# Scale the headway column by 60 to convert raw units (minutes) to hours
# long_swiss_metro["headway_hrs"] = long_swiss_metro["headway"] / 60.0

# Figure out who doesn't incur a marginal cost for the ticket
# This can be because he/she owns an annual season pass (GA == 1) 
# or because his/her employer pays for the ticket (WHO == 2).
# Note that all the other complexity in figuring out ticket costs
# have been accounted for except the GA pass (the annual season
# ticket). Make sure this dummy variable is only equal to 1 for
# the rows with the Train or Swissmetro
# long_swiss_metro["free_ticket"] = (((long_swiss_metro["GA"] == 1) |
#                                     (long_swiss_metro["WHO"] == 2)) &
#                                    long_swiss_metro[custom_alt_id].isin([1,2])).astype(int)
# Scale the travel cost by 100 so estimated coefficients are of similar magnitude
# and acccount for ownership of a season pass
# long_swiss_metro["travel_cost_hundreth"] = (long_swiss_metro["travel_cost"] *
#                                             (long_swiss_metro["free_ticket"] == 0) /
#                                             100.0)

##########
# Create various dummy variables to describe the choice context of a given
# invidual for each choice task.
##########
# Create a dummy variable for whether a person has a single piece of luggage
long_swiss_metro["single_luggage_piece"] = (long_swiss_metro["LUGGAGE"] == 1).astype(int)

# Create a dummy variable for whether a person has multiple pieces of luggage
long_swiss_metro["multiple_luggage_pieces"] = (long_swiss_metro["LUGGAGE"] == 3).astype(int)

# Create a dummy variable indicating that a person is NOT first class
long_swiss_metro["regular_class"] = 1 - long_swiss_metro["FIRST"]

# Create a dummy variable indicating that the survey was taken aboard a train
# Note that such passengers are a-priori imagined to be somewhat partial to train modes
long_swiss_metro["train_survey"] = 1 - long_swiss_metro["SURVEY"]


## Create the model specification
The model specification being used in this example is the following:
$$
\begin{aligned}
V_{i, \textrm{Train}} &= \textrm{ASC Train} + \\
&\quad \beta _{ \textrm{tt_transit} } \textrm{Travel Time} _{ \textrm{Train}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{tc_train} } \textrm{Travel Cost}_{\textrm{Train}} * \left( GA == 0 \right) * 0.01 + \\
&\quad \beta _{ \textrm{headway_train} } \textrm{Headway} _{\textrm{Train}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{survey} } \left( \textrm{Train Survey} == 1 \right) \\
\\
V_{i, \textrm{Swissmetro}} &= \textrm{ASC Swissmetro} + \\
&\quad \beta _{ \textrm{tt_transit} } \textrm{Travel Time} _{ \textrm{Swissmetro}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{tc_sm} } \textrm{Travel Cost}_{\textrm{Swissmetro}} * \left( GA == 0 \right) * 0.01 + \\
&\quad \beta _{ \textrm{headway_sm} } \textrm{Heaway} _{\textrm{Swissmetro}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{seat} } \left( \textrm{Seat Configuration} == 1 \right) \\
&\quad \beta _{ \textrm{survey} } \left( \textrm{Train Survey} == 1 \right) \\
&\quad \beta _{ \textrm{first_class} } \left( \textrm{First Class} == 0 \right) \\
\\
V_{i, \textrm{Car}} &= \beta _{ \textrm{tt_car} } \textrm{Travel Time} _{ \textrm{Car}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{tc_car}} \textrm{Travel Cost}_{\textrm{Car}} * 0.01 + \\
&\quad \beta _{\textrm{luggage}=1} \left( \textrm{Luggage} == 1 \right) + \\
&\quad \beta _{\textrm{luggage}>1} \left( \textrm{Luggage} > 1 \right)
\end{aligned}
$$

Note that packages such as mlogit and statsmodels do not, by default, handle coefficients that vary over some alternatives but not all, such as the travel time coefficient that is specified as being the same for "Train" and "Swissmetro" but different for "Car."

In [41]:
# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or lists of lists of integers. Within a list, 
#         or within the inner-most list, the integers should be the 
#         alternative ID's of the alternative whose utility specification 
#         the explanatory variable is entering. Lists of lists denote 
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [2, 3]
basic_names["intercept"] = ['ASC Transit',
                            'ASC Walk']

basic_specification["travel_time"] = [1, 2, 3]
basic_names["travel_time"] = ['Travel time (Car)',
                              'Travel time (PT)',
                                  'Travel time (Walk)']

# basic_specification["travel_cost_hundreth"] = [1, 2, 3]
# basic_names["travel_cost_hundreth"] = ['Travel Cost * (Annual Pass == 0), units: 0.01 CHF (Train)',
#                                        'Travel Cost * (Annual Pass == 0), units: 0.01 CHF (Swissmetro)',
#                                        'Travel Cost, units: 0.01 CHF (Car)']

# basic_specification["headway_hrs"] = [1, 2]
# basic_names["headway_hrs"] = ["Headway, units:hrs, (Train)",
#                               "Headway, units:hrs, (Swissmetro)"]

basic_specification["Departure time"] = [1, 2, 3]
basic_names["Departure time"] = ['Departure time Car',
                                 'Departure time PT',
                                 'Departure time Walk']

basic_specification["HH size"] = [1, 2, 3]
basic_names["HH size"] = ["HH size Car",
                          "HH size PT",
                          "HH size Walk"]

basic_specification["Nº cars/HH"] = [1, 2, 3]
basic_names["Nº cars/HH"] = ["Nº cars/HH Car", 
                             "Nº cars/HH PT", 
                             "Nº cars/HH Walk"]

basic_specification["Gender"] = [1, 2, 3]
basic_names["Gender"] = ["Gender Car", 
                         "Gender PT", 
                         "Gender Walk"]

basic_specification["Age"] = [1, 2, 3]
basic_names["Age"] = ["Age Car", 
                      "Age PT", 
                      "Age Walk"]

basic_specification["Driver's license"] = [1, 2, 3]
basic_names["Driver's license"] = ["Driver's license Car", 
                                   "Driver's license PT", 
                                   "Driver's license Walk"]

basic_specification["Distance"] = [1, 2, 3]
basic_names["Distance"] = ["Distance Car", 
                           "Distance PT", 
                           "Distance Walk"]


In [ ]:
# NOTE: - Specification and variable names must be ordered dictionaries.
#       - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         of integers or lists of lists of integers. Within a list, 
#         or within the inner-most list, the integers should be the 
#         alternative ID's of the alternative whose utility specification 
#         the explanatory variable is entering. Lists of lists denote 
#         alternatives that will share a common coefficient for the variable
#         in question.

basic_specification = OrderedDict()
basic_names = OrderedDict()

basic_specification["intercept"] = [1, 2]
basic_names["intercept"] = ['ASC Train',
                            'ASC Swissmetro']

basic_specification["travel_time_hrs"] = [[1, 2,], 3]
basic_names["travel_time_hrs"] = ['Travel Time, units:hrs (Train and Swissmetro)',
                                  'Travel Time, units:hrs (Car)']

basic_specification["travel_cost_hundreth"] = [1, 2, 3]
basic_names["travel_cost_hundreth"] = ['Travel Cost * (Annual Pass == 0), units: 0.01 CHF (Train)',
                                       'Travel Cost * (Annual Pass == 0), units: 0.01 CHF (Swissmetro)',
                                       'Travel Cost, units: 0.01 CHF (Car)']

basic_specification["headway_hrs"] = [1, 2]
basic_names["headway_hrs"] = ["Headway, units:hrs, (Train)",
                              "Headway, units:hrs, (Swissmetro)"]

basic_specification["seat_configuration"] = [2]
basic_names["seat_configuration"] = ['Airline Seat Configuration, base=No (Swissmetro)']

basic_specification["train_survey"] = [[1, 2]]
basic_names["train_survey"] = ["Surveyed on a Train, base=No, (Train and Swissmetro)"]

basic_specification["regular_class"] = [1]
basic_names["regular_class"] = ["First Class == False, (Swissmetro)"]

basic_specification["single_luggage_piece"] = [3]
basic_names["single_luggage_piece"] = ["Number of Luggage Pieces == 1, (Car)"]

basic_specification["multiple_luggage_pieces"] = [3]
basic_names["multiple_luggage_pieces"] = ["Number of Luggage Pieces > 1, (Car)"]


# Estimate the conditional logit model

In [42]:
# Estimate the multinomial logit model (MNL)
trips_mnl = pl.create_choice_model(data=long_trips,
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

# Specify the initial values and method for the optimization.
trips_mnl.fit_mle(np.zeros(26))

# Look at the estimation results
trips_mnl.get_statsmodels_summary()

Log-likelihood at zero: -11,335.4816
Initial Log-likelihood: -11,335.4816


c:\Users\iazka\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 0.66 seconds.
Final log-likelihood: -3,759.5913


c:\Users\iazka\anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1228: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.cov)),
c:\Users\iazka\anaconda3\lib\site-packages\pylogit\base_multinomial_cm_v2.py:1259: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.robust_cov)),


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                        Modo   No. Observations:               10,318
Model:             Multinomial Logit Model   Df Residuals:                   10,292
Method:                                MLE   Df Model:                           26
Date:                     Wed, 08 May 2024   Pseudo R-squ.:                   0.668
Time:                             12:00:41   Pseudo R-bar-squ.:               0.666
AIC:                             7,571.183   Log-Likelihood:             -3,759.591
BIC:                             7,759.465   LL-Null:                   -11,335.482
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
ASC Transit               0.0516      0.324      0.159      0.873      -0.583       0.687
ASC Walk                  4.4767      0.447     10.010      0.000       3.600       5.353
Travel time (Car)         0.2717      0.020     13.879      0.000       0.233       0.310
Travel time (PT)         -0.1185      0.005    -24.873      0.000      -0.128      -0.109
Travel time (Walk)       -0.1710      0.010    -16.723      0.000      -0.191      -0.151
Departure time Car       -0.0013        nan        nan        nan         nan         nan
Departure time PT         0.0013        nan        nan        nan         nan         nan
Departure time Walk    6.188e-05        nan        nan        nan         nan         nan
HH size Car               0.0082        nan        nan        nan         nan         nan
HH size PT                0.0026        nan        nan        nan         nan         nan
HH size Walk             -0.0109        nan        nan        nan         nan         nan
Nº cars/HH Car            0.6358   1.82e+05    3.5e-06      1.000   -3.56e+05    3.56e+05
Nº cars/HH PT            -0.4500   1.82e+05  -2.48e-06      1.000   -3.56e+05    3.56e+05
Nº cars/HH Walk          -0.1858   1.82e+05  -1.02e-06      1.000   -3.56e+05    3.56e+05
Gender Car               -0.8117        nan        nan        nan         nan         nan
Gender PT                 0.8070        nan        nan        nan         nan         nan
Gender Walk               0.0047        nan        nan        nan         nan         nan
Age Car                  -0.0195        nan        nan        nan         nan         nan
Age PT                   -0.0315        nan        nan        nan         nan         nan
Age Walk                  0.0510        nan        nan        nan         nan         nan
Driver's license Car     -1.3926        nan        nan        nan         nan         nan
Driver's license PT       0.8582        nan        nan        nan         nan         nan
Driver's license Walk     0.5344        nan        nan        nan         nan         nan
Distance Car             -0.0001        nan        nan        nan         nan         nan
Distance PT               0.0002        nan        nan        nan         nan         nan
Distance Walk          -6.18e-05        nan        nan        nan         nan         nan
=========================================================================================
"""

## View results without using statsmodels summary table

You can view all of the results simply by using print_summaries(). This will simply print the various summary dataframes.

In [43]:
# Look at other all results at the same time
trips_mnl.print_summaries()



Number of Parameters                                                     26
Number of Observations                                                10318
Null Log-Likelihood                                           -11335.481594
Fitted Log-Likelihood                                          -3759.591269
Rho-Squared                                                        0.668334
Rho-Bar-Squared                                                    0.666041
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object
                       parameters        std_err    t_stats       p_values  \
ASC Transit              0.051602       0.324017   0.159256   8.734674e-01   
ASC Walk                 4.476678       0.447228  10.009832   1.379860e-23   
Travel time (Car)        0.271713       0.019577  13.879423   8.442057e-44   
Travel time (PT)        -0.118511       0.004765 -24.873303  1.447410e-136   
Travel time (Walk)      -0.170952       0.010222 -16.723406   

In [11]:
# Look at the general and goodness of fit statistics
swissmetro_mnl.fit_summary

Number of Parameters                                                     14
Number of Observations                                                 6768
Null Log-Likelihood                                               -6964.663
Fitted Log-Likelihood                                             -5159.258
Rho-Squared                                                       0.2592236
Rho-Bar-Squared                                                   0.2572134
Estimation Message        Desired error not necessarily achieved due to ...
dtype: object

In [12]:
# Look at the parameter estimation results, and round the results for easy viewing
np.round(swissmetro_mnl.summary, 3)

,parameters,std_err,t_stats,p_values,robust_std_err,robust_t_stats,robust_p_values
ASC Train,-1.293,0.146,-8.845,0.000,0.303,-4.274,0.000
ASC Swissmetro,-0.503,0.116,-4.332,0.000,0.392,-1.281,0.200
"Travel Time, units:hrs (Train and Swissmetro)",-0.699,0.042,-16.545,0.000,0.146,-4.772,0.000
"Travel Time, units:hrs (Car)",-0.723,0.047,-15.340,0.000,0.164,-4.398,0.000
"Travel Cost * (Annual Pass == 0), units: 0.01 CHF (Train)",-0.562,0.094,-6.002,0.000,0.129,-4.359,0.000
"Travel Cost * (Annual Pass == 0), units: 0.01 CHF (Swissmetro)",-0.282,0.045,-6.252,0.000,0.067,-4.236,0.000
"Travel Cost, units: 0.01 CHF (Car)",-0.514,0.104,-4.953,0.000,0.230,-2.234,0.025
"Headway, units:hrs, (Train)",-0.314,0.062,-5.063,0.000,0.061,-5.137,0.000
"Headway, units:hrs, (Swissmetro)",-0.377,0.196,-1.925,0.054,0.207,-1.827,0.068
"Airline Seat Configuration, base=No (Swissmetro)",-0.782,0.087,-8.970,0.000,0.097,-8.058,0.000
